# Data Prep for ATP 

**Duplicate notebook using concat method**

* manual cleaning needed outside of notebook for funding data

In [1]:
import intake
import numpy as np
import pandas as pd
from calitp import to_snakecase
from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *

import altair as alt

import data_cleaning

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
pd.set_option("display.max_columns",500)

* some `a1_imp_agcy_fed_ma_num` are not present 
*  merge on `project_app_id`


* need function for reading in funding data and which projects get selected for funding
* sheets of Master_AllData that we need:
    * Master_Yes
    * Statewide SUR Details (merge with SUR Funding
    * Statewide SUR Funding (merge with SUR Details) 
* using a copy of the data to account for multiple headers

In [3]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


## Read in Master Data

In [4]:
# reading the clean data (from atp script)
#df = data_cleaning.read_clean_data()

In [5]:
#df.sample()

In [6]:
def read_app_data():
    """
    Function for reading in the application data. Can then merge with funded data
    """
    # identify information columns that we need to drop
    columns_to_drop = ['a1_imp_agcy_contact','a1_imp_agcy_email','a1_imp_agcy_phone',
                      'a1_proj_partner_contact', 'a1_proj_partner_email', 'a1_proj_partner_phone']
    
    #read in data
    df = to_snakecase(pd.read_excel(f'{GCS_FILE_PATH}Master_AllData_Cycle5_Field_Mapping_COPY.xls',
              sheet_name='Master_Yes',
                                        header=[2]))
    df = df.drop(columns = columns_to_drop)
    
    #drop columns that will contain funding data (this df has the columns but no information for them)
    #we know funding data starts with columns `original_prog__amt___pa_ed_`
    df.drop(df.iloc[:,(df.columns.get_loc('original_prog__amt___pa_ed_')):], inplace=True, axis=1)  
    
    #drop identifier columns that are fully null (these columns are populated in funding data
    df = df.drop(columns={'#', 'atp_id', 'ppno', 'ppno_1'})
    
    return df

In [7]:
## cn use the version provided or the one we cleaned in the cleaning script
master_data2= read_app_data()

In [8]:
master_data =  data_cleaning.read_clean_data()


2022-10-26 15:07:16.712 | INFO     | data_cleaning:export_district_need_assistance:119 - got District matches: There are 25 Legislative District entries needing assistance
2022-10-26 at 15:07:16 | INFO | got District matches: There are 25 Legislative District entries needing assistance


There are 25 Legislative District entries needing assistance


/home/jovyan/data-analyses/dla/atp/data_cleaning.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
2022-10-26 15:07:17.876 | INFO     | data_cleaning:check_counties:182 - got Locode matches: There are 7 Locode Matching entries needing assistance
2022-10-26 at 15:07:17 | INFO | got Locode matches: There are 7 Locode Matching entries needing assistance


There are 7 Locode Matching entries needing assistance


/home/jovyan/data-analyses/dla/atp/data_cleaning.py:205: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
/home/jovyan/data-analyses/dla/atp/data_cleaning.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
2022-10-26 15:07:18.491 | INFO     | data_cleaning:find_potential_locode_matches:238 - got potential Locode matches: There are 8 potential locode matching entries needing assistance
2022-10-26 at 15:07:18 | INFO | got potential Locode matches: There are 8 potential locode matching entries needing assistance


There are 8 potential locode matching entries needing assistance


2022-10-26 15:07:24.425 | INFO     | data_cleaning:clean_data:286 - got clean data: 882 entries
2022-10-26 at 15:07:24 | INFO | got clean data: 882 entries


Data cleaning complete. There are 882 entries in dataframe


In [9]:
master_data.head()

,a1_imp_agcy_city,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_name,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_agcy,a1_proj_partner_exists,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_rout

In [10]:
len(list(master_data.columns))

218

## Function to read SUR funding data

In [11]:
def read_SUR_funding_data():
    """
    Function to read in ATP funding data. Function will need to change for future data.
    Notes:
    * `atp_id` columns appear the same but the sur_details has an extra zero in the middle of the string so it would not match
    * `a3_project_type` also is entered differently however, details has more details than the funding sheet. Has information on size of project. can add to new column
    * `a1_imp_agcy_name_x` has manual errors so selecting `a1_imp_agcy_name_y`
    """
    # identify information columns that we need to drop
    columns_to_drop = ['a1_imp_agcy_contact','a1_imp_agcy_email','a1_imp_agcy_phone',
                      'a1_proj_partner_contact', 'a1_proj_partner_email', 'a1_proj_partner_phone']
    #read in SUR details and SUR funding data
    sur_details = to_snakecase(pd.read_excel(f'{GCS_FILE_PATH}Master_AllData_Cycle5_Field_Mapping_COPY.xls',
              sheet_name='Statewide SUR Details'))
    sur_details = sur_details.drop(columns = columns_to_drop)
    
    sur_funding = to_snakecase(pd.read_excel(f'{GCS_FILE_PATH}Master_AllData_Cycle5_Field_Mapping_COPY.xls',
              sheet_name='Statewide SUR Funding'))
    
    #drop the last few columns of SUR Details that have no funding data entered, but have columns
    sur_details.drop(sur_details.iloc[:,199:], inplace=True, axis=1)
    
    #remove rows with all null values
    cols_to_check = sur_funding.columns
    sur_funding['is_na'] = sur_funding[cols_to_check].isnull().apply(lambda x: all(x), axis=1) 
    sur_funding = sur_funding>>filter(_.is_na==False)
    sur_funding = sur_funding.drop(columns={'is_na'})

    #delete rows identified that are not part of the data (informational cells) or a sum total for all entries
    delete_row = sur_funding[sur_funding["project_cycle"]== 'Added Field not from App'].index
    sur_funding = sur_funding.drop(delete_row)
    
    delete_row = sur_funding[sur_funding["total_project_cost"]== '370,984,000.00'].index
    sur_funding = sur_funding.drop(delete_row)
    
    #merge sur_funding and sur_details
    merge_on = ['project_app_id', 'project_cycle', 'a2_ct_dist', 'a1_locode']
    df = (pd.merge(sur_details, sur_funding, how="outer", on = merge_on, indicator=True))
    
    #keep entries that merge. Right_only rows are misentered and more informational columns  
    df = df>>filter(_._merge=='both')
    
    # filling the null values for some of the duplicate columns
    # manually checking that values are the same as of now- will add function to check when we get the data links
    df['awarded_x'] = df['awarded_x'].fillna(df['awarded_y'])
    df['ppno_y'] = df['ppno_y'].fillna(df['ppno_x'])
    
    #renaming and dropping duplicate columns 
    ## a1_imp_agcy_name_x has manual errors so selecting a1_imp_agcy_name_y
    df = df.rename(columns={'awarded_x':'awarded',
                                'ppno_y':'ppno',
                                'a1_imp_agcy_name_y':'a1_imp_agcy_name',
                                'a2_info_proj_name_y':'a2_info_proj_name'
                               })
    df = df.drop(columns={'awarded_y', 'a1_imp_agcy_name_x', 'a2_info_proj_name_x','ppno_x', '_merge'})
    df["data_origin"]="Funded"
    
    return df

In [12]:
funded = read_SUR_funding_data()

/tmp/ipykernel_2084/3437006353.py:38: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
/tmp/ipykernel_2084/3437006353.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_2084/3437006353.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
len(funded)

49

In [14]:
len(list(funded.columns))

264

In [15]:
funded.head()

,awarded,project_cycle,a2_ct_dist,#,atp_id_x,ppno_1,a3_proj_type_x,project_app_id,a2_county,a1_locode,a1_imp_agcy_street,a1_imp_agcy_city,a1_imp_agcy_zip,a1_imp_agcy_title,a1_imp_agcy_ma,a1_imp_agcy_state_ma_num,a1_imp_agcy_fed_ma_num,a1_proj_partner_exists,a1_proj_partner_agcy,a1_proj_partner_title,assembly_district,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,congressional_district,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,senate_district,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a2_info_proj_descr,a2_info_proj_loc,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_current_plan,a3_trails,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,agency_fully_own_r_w,"require_r_w_easement,_from_gov",_require_rw_easement_from_private,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_routes,a4_row_open_street_demo,project_status,solicitation_abv,solicitation,soliciting_agency,atp_id_y,ppno,ppno1,oversight_id,a3_proj_type_y,project_size,total_project_cost,a1_imp_agcy_name,a2_info_proj_name,original_prog__amt___pa_ed_,orig__pr

In [16]:
import re

In [17]:
dcolumns1 = [col for col in funded.columns if isinstance(col, str) and re.match('.*_x', col)]

In [18]:
dcolumns2 = [col for col in funded.columns if isinstance(col, str) and re.match('.*_y', col)]

In [19]:
dcolumns1

['atp_id_x',
 'a3_proj_type_x',
 'match_total_atp_$_and_total_atp_x_1000',
 'total_atp_x_1000']

In [20]:
dcolumns2

['atp_id_y',
 'a3_proj_type_y',
 'orig__prog__year__pa_ed_',
 'orig__prog__year__ps_e_',
 'orig__prog__year__rw_',
 'orig__prog__year__con_',
 'orig__prog__year__con_ni_',
 'fund_year_1',
 'fund_year_2',
 'fund_year_3',
 'fund_year_4']

In [21]:
funded>>select(_.atp_id_x, _.atp_id_y,
               _.a3_proj_type_x, _.a3_proj_type_y)

,atp_id_x,atp_id_y,a3_proj_type_x,a3_proj_type_y
0,ATP5-05-001R,ATP5-5-001R,Infrastructure - Small,Infrastructure
1,ATP5-07-002S,ATP5-7-002S,Infrastructure - Small,Infrastructure
2,ATP5-04-003S,ATP5-4-003S,Infrastructure + NI - Large,Combined (IF and NI)
3,ATP5-05-004S,ATP5-5-004S,Infrastructure + NI - Large,Combined (IF and NI)
4,ATP5-11-005S,ATP5-11-005S,Infrastructure + NI - Small,Combined (IF and NI)
5,ATP5-06-006S,ATP5-6-006S,Infrastructure + NI - Small,Combined (IF and NI)
6,ATP5-10-007S,ATP5-10-007S,Infrastructure - Small,Infrastructure
7,ATP5-05-008S,ATP5-5-008S,Infrastructure - Large,Infrastructure
8,ATP5-02-009R,ATP5-2-009R,Infrastructure - Large,Infrastructure
9,ATP5-06-010S,ATP5-6-010S,Infrastructure + NI - Small,Combined (IF and NI)


## Merging with Master_Data

In [22]:
master_data.sample()

,a1_imp_agcy_city,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_name,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_agcy,a1_proj_partner_exists,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_rout

In [23]:
funded.sample()

,awarded,project_cycle,a2_ct_dist,#,atp_id_x,ppno_1,a3_proj_type_x,project_app_id,a2_county,a1_locode,a1_imp_agcy_street,a1_imp_agcy_city,a1_imp_agcy_zip,a1_imp_agcy_title,a1_imp_agcy_ma,a1_imp_agcy_state_ma_num,a1_imp_agcy_fed_ma_num,a1_proj_partner_exists,a1_proj_partner_agcy,a1_proj_partner_title,assembly_district,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,congressional_district,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,senate_district,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a2_info_proj_descr,a2_info_proj_loc,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_current_plan,a3_trails,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,agency_fully_own_r_w,"require_r_w_easement,_from_gov",_require_rw_easement_from_private,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_routes,a4_row_open_street_demo,project_status,solicitation_abv,solicitation,soliciting_agency,atp_id_y,ppno,ppno1,oversight_id,a3_proj_type_y,project_size,total_project_cost,a1_imp_agcy_name,a2_info_proj_name,original_prog__amt___pa_ed_,orig__pr

In [24]:
master_data.loc[master_data['project_cycle'] == 'CYCLE 5', 'project_cycle'] = 5
master_data.loc[master_data['project_cycle'] == 'CYCLE 6', 'project_cycle'] = 6

In [25]:
master_data[['a2_ct_dist','project_cycle']] = master_data[['a2_ct_dist','project_cycle']].astype(int)

In [26]:
master_data[['a1_locode']] = master_data[['a1_locode']].astype(object)

In [27]:
subset = master_data>>select(_.project_app_id, _.project_cycle, _.a2_ct_dist, _.a1_locode)

In [28]:
subset.project_cycle.value_counts()

5    448
6    434
Name: project_cycle, dtype: int64

In [29]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 881
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   project_app_id  882 non-null    object
 1   project_cycle   882 non-null    int64 
 2   a2_ct_dist      882 non-null    int64 
 3   a1_locode       868 non-null    object
dtypes: int64(2), object(2)
memory usage: 34.5+ KB


In [30]:
(master_data2>>select(_.project_app_id, _.project_cycle, _.a2_ct_dist, _.a1_locode)).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   project_app_id  454 non-null    object
 1   project_cycle   454 non-null    int64 
 2   a2_ct_dist      454 non-null    int64 
 3   a1_locode       453 non-null    object
dtypes: int64(2), object(2)
memory usage: 14.3+ KB


In [31]:
(funded.iloc[:,0:20]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   awarded                   49 non-null     object 
 1   project_cycle             49 non-null     object 
 2   a2_ct_dist                49 non-null     float64
 3   #                         0 non-null      float64
 4   atp_id_x                  49 non-null     object 
 5   ppno_1                    8 non-null      object 
 6   a3_proj_type_x            49 non-null     object 
 7   project_app_id            49 non-null     object 
 8   a2_county                 49 non-null     object 
 9   a1_locode                 49 non-null     object 
 10  a1_imp_agcy_street        49 non-null     object 
 11  a1_imp_agcy_city          49 non-null     object 
 12  a1_imp_agcy_zip           49 non-null     float64
 13  a1_imp_agcy_title         49 non-null     object 
 14  a1_imp_agcy_

In [32]:
merge_on = ['project_app_id', 'project_cycle', 'a2_ct_dist', 'a1_locode']

In [33]:
#pd.concat([subset,funded])

In [34]:
c1 = subset.select_dtypes(np.float).columns
c1

/tmp/ipykernel_2084/3478909441.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Index([], dtype='object')

In [35]:
# will have suplicates for atp_id_x a3_proj_type_x
(pd.merge(subset, funded, how='outer', on= merge_on, indicator='full_merge')).head()

/tmp/ipykernel_2084/2365909036.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


,project_app_id,project_cycle,a2_ct_dist,a1_locode,awarded,#,atp_id_x,ppno_1,a3_proj_type_x,a2_county,a1_imp_agcy_street,a1_imp_agcy_city,a1_imp_agcy_zip,a1_imp_agcy_title,a1_imp_agcy_ma,a1_imp_agcy_state_ma_num,a1_imp_agcy_fed_ma_num,a1_proj_partner_exists,a1_proj_partner_agcy,a1_proj_partner_title,assembly_district,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,congressional_district,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,senate_district,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a2_info_proj_descr,a2_info_proj_loc,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_current_plan,a3_trails,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,agency_fully_own_r_w,"require_r_w_easement,_from_gov",_require_rw_easement_from_private,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_routes,a4_row_open_street_demo,project_status,solicitation_abv,solicitation,soliciting_agency,atp_id_y,ppno,ppno1,oversight_id,a3_proj_type_y,project_size,total_project_cost,a1_imp_agcy_name,a2_info_proj_name,original_prog__amt___pa_ed_,orig__pr

In [36]:
(pd.merge(subset, funded, how='outer', on= merge_on, indicator='full_merge')).full_merge.value_counts()

/tmp/ipykernel_2084/2300597696.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


left_only     882
right_only     49
both            0
Name: full_merge, dtype: int64

In [37]:
(pd.merge(subset, funded, how='outer', on= merge_on, indicator='full_merge')).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931 entries, 0 to 930
Columns: 265 entries, project_app_id to full_merge
dtypes: category(1), float64(155), object(109)
memory usage: 1.9+ MB


/tmp/ipykernel_2084/3448293844.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


In [38]:
# subset data in app data (will concat data later) and keep full funded columns

### test using merge

In [39]:
#(pd.merge(master_data, funded, how='outer', on= merge_on, indicator='full_merge')).head()

In [40]:
#(pd.merge(master_data, funded, how='outer', on=merge_on2, indicator='full_merge')).full_merge.value_counts()

In [41]:
master_data.columns.intersection(funded.columns)

Index(['a1_imp_agcy_city', 'a1_imp_agcy_fed_ma_num', 'a1_imp_agcy_ma',
       'a1_imp_agcy_name', 'a1_imp_agcy_state_ma_num', 'a1_imp_agcy_street',
       'a1_imp_agcy_title', 'a1_imp_agcy_zip', 'a1_proj_partner_agcy',
       'a1_proj_partner_exists',
       ...
       'a4_plan_school_routes', 'a4_row_open_street_demo', 'assembly_district',
       'congressional_district', 'senate_district', '#', 'awarded', 'ppno',
       'ppno_1', 'data_origin'],
      dtype='object', length=195)

In [42]:
master_data.columns.difference(funded.columns)

Index(['a1_letter_of_intent', 'a3_proj_type', 'a4_row_100', 'a4_row_gov_ease',
       'a4_row_private_ease', 'agency_app_num', 'app_fk', 'app_pk', 'atp_id',
       'attch_addtl_attachments', 'attch_app_sig_page',
       'attch_conditions_photos', 'attch_conditions_project_map',
       'attch_engineeers_checklist', 'attch_exhibit22_plan',
       'attch_letters_of_support', 'attch_link', 'attch_ni_workplan',
       'attch_project_estimate', 'completed_pdf_form',
       'details_datetime_stamp', 'geometry', 'main_datetime_stamp'],
      dtype='object')

In [43]:
funded.columns.difference(master_data.columns)

Index(['_2122_con', '_2122_con_ni', '_2122_pa_ed', '_2122_ps_e', '_2122_rw',
       '_2122_total', '_2223_con', '_2223_con_ni', '_2223_pa_ed', '_2223_ps_e',
       '_2223_rw', '_2223_total', '_2324_con', '_2324_con_ni', '_2324_pa_ed',
       '_2324_ps_e', '_2324_rw', '_2324_total', '_2425_con', '_2425_con_ni',
       '_2425_pa_ed', '_2425_ps_e', '_2425_rw', '_2425_total',
       '_require_rw_easement_from_private', 'a3_proj_type_x', 'a3_proj_type_y',
       'agency_fully_own_r_w', 'atp_id_x', 'atp_id_y', 'con', 'con_ni',
       'fund_year_1', 'fund_year_2', 'fund_year_3', 'fund_year_4',
       'match_total_atp_$_and_total', 'match_total_atp_$_and_total_atp_x_1000',
       'orig__prog__year__con_', 'orig__prog__year__con_ni_',
       'orig__prog__year__pa_ed_', 'orig__prog__year__ps_e_',
       'orig__prog__year__rw_', 'original_prog__amt___con_ni_',
       'original_prog__amt___pa_ed_', 'original_prog__amt___ps_e_',
       'original_prog__amt___rw_', 'orignal_prog__amt___con_', 'oversi

In [44]:
# # merging on all common columns does not work
# merge_on3 = list(master_data.columns.intersection(funded.columns))

* merging on [`project_app_id`, `project_cycle`, `a2_ct_dist`, `a1_locode`] 

In [45]:
merge_on3 = ['project_app_id', 'project_cycle', 'a2_ct_dist', 
             # 'a1_locode', 'a2_county', 'a2_info_proj_name',
             # 'a1_imp_agcy_name', 
              # 'a1_imp_agcy_street', 'a1_imp_agcy_city',
 # 'a1_imp_agcy_zip', 'a1_imp_agcy_title', 'a1_imp_agcy_ma', 'a1_imp_agcy_state_ma_num',
 # 'a1_imp_agcy_fed_ma_num', 'a1_proj_partner_exists', 'a1_proj_partner_agcy', 'a1_proj_partner_title',
 #'assembly_district', 'a2_assem_dist_a', 'a2_assem_dist_b', 'a2_assem_dist_c',
 #'congressional_district', 'a2_congress_dist_a', 'a2_congress_dist_b', 'a2_congress_dist_c', 'senate_district',
 #'a2_senate_dist_a', 'a2_senate_dist_b', 'a2_senatedistc', 'a2_info_proj_loc',
  # 'a2_mop_uza_population', 'a2_mpo', 'a2_past_proj', 'a2_past_proj_qty',
  # 'a2_proj_lat', 'a2_proj_long', 'a2_proj_scope_summary',
  # 'a2_project_location_map', 'a2_rtpa', 'a3_plan_active_trans', 'a3_plan_active_trans_exists',
  # 'a3_plan_bicycle', 'a3_plan_bicycle_exists', 'a3_plan_ped', 'a3_plan_ped_exists',
  # 'a3_plan_srts', 'a3_plan_srts_exists', 'a3_st_bicycle_applies',
  # 'a3_st_bicycle_pct', 'a3_st_num_schools', 'a3_st_ped_applies',
   # 'a3_st_ped_pct', 'a3_st_srts', 'a3_trail_elig_cost', 'a3_trail_fed_funding',
]

In [46]:
#(master_data2[master_data2.columns.intersection(merge_on3)]).info()

In [47]:
#(master_data[master_data.columns.intersection(merge_on3)]).info()

In [48]:
dfall = (pd.merge(master_data, funded, how='outer', on=merge_on3, indicator='full_merge'))

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/tmp/ipykernel_2084/2647171281.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


In [49]:
dfall.full_merge.value_counts()

left_only     835
both           47
right_only      2
Name: full_merge, dtype: int64

In [50]:
compare_entries = np.where(dfall["a1_imp_agcy_name_x"] == dfall["a1_imp_agcy_name_y"], True, False)
dfall["compare_desc"] = compare_entries

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [51]:
dfall.compare_desc.value_counts()

False    838
True      46
Name: compare_desc, dtype: int64

In [104]:
dfall>>filter(_.compare_desc==False)>>select(_.a1_imp_agcy_name_x,  _.a1_imp_agcy_name_y)>>arrange(_.a1_imp_agcy_name_y)

,a1_imp_agcy_name_x,a1_imp_agcy_name_y
882,NaN,"Maywood, City of"
883,NaN,"Oakland, City of"
33,"South El Monte, City of-4","South El Monte, City of"
0,Merced County,NaN
1,"Santa Ana, City of",NaN
...,...,...
877,"Thousand Oaks, City of",NaN
878,"South El Monte, City of",NaN
879,"San Jose, City of",NaN
880,Santa Barbara County,NaN


In [53]:
dfall>>filter(_.full_merge=='right_only')

,a1_imp_agcy_city_x,a1_imp_agcy_fed_ma_num_x,a1_imp_agcy_ma_x,a1_imp_agcy_name_x,a1_imp_agcy_state_ma_num_x,a1_imp_agcy_street_x,a1_imp_agcy_title_x,a1_imp_agcy_zip_x,a1_letter_of_intent,a1_proj_partner_agcy_x,a1_proj_partner_exists_x,a1_proj_partner_title_x,a2_assem_dist_a_x,a2_assem_dist_b_x,a2_assem_dist_c_x,a2_congress_dist_a_x,a2_congress_dist_b_x,a2_congress_dist_c_x,a2_county_x,a2_ct_dist,a2_info_proj_descr_x,a2_info_proj_loc_x,a2_info_proj_name_x,a2_mop_uza_population_x,a2_mpo_x,a2_past_proj_x,a2_past_proj_qty_x,a2_proj_lat_x,a2_proj_long_x,a2_proj_scope_summary_x,a2_project_location_map_x,a2_rtpa_x,a2_senate_dist_a_x,a2_senate_dist_b_x,a2_senatedistc_x,a3_plan_active_trans_x,a3_plan_active_trans_exists_x,a3_plan_bicycle_x,a3_plan_bicycle_exists_x,a3_plan_ped_x,a3_plan_ped_exists_x,a3_plan_srts_x,a3_plan_srts_exists_x,a3_proj_type,a3_st_bicycle_applies_x,a3_st_bicycle_pct_x,a3_st_num_schools_x,a3_st_ped_applies_x,a3_st_ped_pct_x,a3_st_srts_x,a3_trail_elig_cost_x,a3_trail_fed_funding_x,a3_trail_trans_pct_x,a3_trails_x,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode_x,a3_plan_none_x,a3_plan_other_x,a3_plan_other_desc_x,a2_output_outcome_x,a3_current_plan_x,b_sig_inter_new_bike_boxes_x,b_class_1_x,b_class_2_x,b_class_3_x,b_class_4_x,a4_bike_gap_pct_x,b_light_intersection_x,b_mid_block_new_rrfb_signal_x,b_mid_block_surf_improv_x,b_bsp_new_bikes_x,b_bike_new_secured_lockers_x,b_bike_new_racks_x,b_bsp_new_station_x,b_other_bike_improv_1_x,b_other_bike_improv_qty_1_x,b_other_bike_improv_2_x,b_other_bike_improv_qty_2_x,b_light_rdwy_seg_x,b_sig_inter_timing_improv_x,b_un_sig_new_rrfb_signal_x,b_un_sig_cross_surf_improv_x,a4_easement_support_x,m_cls_1_trails_widen_recon_exist_x,m_cls_1_trails_new__less_than_9_x,m_cls_1_trails_new_over_9_x,m_non_cls_trails_new_x,m_other_trail_imprv_1_x,m_other_trail_improv_qty_1_x,m_other_trail_imprv_2_x,m_other_trail_improv_qty_2_x,m_non_cls_widen_recon_exist_x,p_amenities_bench_x,a4_ped_gap_pct_x,p_mid_block_cross_new_rrfb_signal_x,p_light_intersection_x,p_lighting_rdwy_seg_x,p_mid_block_cross_surf_improv_x,p_new_ada_ramp_x,p_sidewlks_new_barrier_protect_x,p_sidewlks_new_4_to_8_x,p_sidewlks_new_over_8_x,p_other_ped_imprv_1_x,p_other_ped_qty_1_x,p_other_ped_imprv_2_x,p_other_ped_qty_2_x,p_reconstruct_ramp_to_ada_stand_x,p_sidewlks_reconstruct_enhance_exist_x,p_sig_inter_enhance_exist_crosswlk_x,p_sig_inter_new_crosswlk_x,p_sig_inter_ped_heads_x,p_sig_inter_shorten_cross_x,p_sig_inter_timing_improv_x,p_amenities_trash_can_x,p_amenities_shade_tree_x,p_amenities_shade_tree_type_x,p_un_sig_inter_new_traff_sig_x,p_un_sig_inter_new_roundabout_x,p_un_sig_inter_new_rrfb_sig_x,p_un_sig_inter_shorten_cross_x,p_un_sig_inter_cross_surface_improv_x,p_sidewlks_widen_existing_x,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1_x,v_speed_feedback_signs_x,v_other_traffic_calming_qty_1_x,v_other_traffic_calming_imprv_2_x,v_other_traffic_calming_qty_2_x,v_remove_right_turn_pocket_x,v_remove_travel_ln_x,v_sig_inter_new_roundabout_x,v_sig_inter_timing_improv_x,v_un_sig_inter_new_traf_sig_x,v_un_sig_inter_new_roundabout_x,app_fk,details_datetime_stamp,a4_reg_init_x,a4_reg_init_pct_x,a4_com_init_x,a4_com_init_pct_x,a4_safe_route_x,a4_safe_route_pct_x,a4_fl_mile_x,a4_fl_mile_pct_x,a4_emp_based_x,a4_emp_based_pct_x,a4_other_ni_x,a4_other_ni_descr_x,a4_other_ni_pct_x,a4_wb_audits_x,a4_bike_classes_x,a4_ped_classes_x,a4_demo_events_x,a4_com_enc_x,a4_le_methods_x,a4_com_meetings_x,a4_classrooms_x,a4_school_assem_x,a4_after_school_x,a4_bike_rodeos_x,a4_mock_cities_x,a4_walk_bus_x,a4_bike_train_x,a4_com_challenges_x,a4_srts_enc_x,a4_srts_le_x,a4_srts_training_x,a4_act_other_1_x,a4_act_other_1_descr_x,a4_act_other_2_x,a4_act_othe

In [54]:
dfall.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 884 entries, 0 to 883
Columns: 481 entries, a1_imp_agcy_city_x to compare_desc
dtypes: Int64(9), bool(1), category(1), datetime64[ns](2), float64(265), geometry(1), object(202)
memory usage: 3.2+ MB


In [55]:
# columns in common with the dfs we merged
list(master_data.columns.intersection(funded.columns))

['a1_imp_agcy_city',
 'a1_imp_agcy_fed_ma_num',
 'a1_imp_agcy_ma',
 'a1_imp_agcy_name',
 'a1_imp_agcy_state_ma_num',
 'a1_imp_agcy_street',
 'a1_imp_agcy_title',
 'a1_imp_agcy_zip',
 'a1_proj_partner_agcy',
 'a1_proj_partner_exists',
 'a1_proj_partner_title',
 'a2_assem_dist_a',
 'a2_assem_dist_b',
 'a2_assem_dist_c',
 'a2_congress_dist_a',
 'a2_congress_dist_b',
 'a2_congress_dist_c',
 'a2_county',
 'a2_ct_dist',
 'a2_info_proj_descr',
 'a2_info_proj_loc',
 'a2_info_proj_name',
 'a2_mop_uza_population',
 'a2_mpo',
 'a2_past_proj',
 'a2_past_proj_qty',
 'a2_proj_lat',
 'a2_proj_long',
 'a2_proj_scope_summary',
 'a2_project_location_map',
 'a2_rtpa',
 'a2_senate_dist_a',
 'a2_senate_dist_b',
 'a2_senatedistc',
 'a3_plan_active_trans',
 'a3_plan_active_trans_exists',
 'a3_plan_bicycle',
 'a3_plan_bicycle_exists',
 'a3_plan_ped',
 'a3_plan_ped_exists',
 'a3_plan_srts',
 'a3_plan_srts_exists',
 'a3_st_bicycle_applies',
 'a3_st_bicycle_pct',
 'a3_st_num_schools',
 'a3_st_ped_applies',
 'a3_

In [56]:
# (dfall.iloc[:,199:230]).info()

In [57]:
dfall.sample()

,a1_imp_agcy_city_x,a1_imp_agcy_fed_ma_num_x,a1_imp_agcy_ma_x,a1_imp_agcy_name_x,a1_imp_agcy_state_ma_num_x,a1_imp_agcy_street_x,a1_imp_agcy_title_x,a1_imp_agcy_zip_x,a1_letter_of_intent,a1_proj_partner_agcy_x,a1_proj_partner_exists_x,a1_proj_partner_title_x,a2_assem_dist_a_x,a2_assem_dist_b_x,a2_assem_dist_c_x,a2_congress_dist_a_x,a2_congress_dist_b_x,a2_congress_dist_c_x,a2_county_x,a2_ct_dist,a2_info_proj_descr_x,a2_info_proj_loc_x,a2_info_proj_name_x,a2_mop_uza_population_x,a2_mpo_x,a2_past_proj_x,a2_past_proj_qty_x,a2_proj_lat_x,a2_proj_long_x,a2_proj_scope_summary_x,a2_project_location_map_x,a2_rtpa_x,a2_senate_dist_a_x,a2_senate_dist_b_x,a2_senatedistc_x,a3_plan_active_trans_x,a3_plan_active_trans_exists_x,a3_plan_bicycle_x,a3_plan_bicycle_exists_x,a3_plan_ped_x,a3_plan_ped_exists_x,a3_plan_srts_x,a3_plan_srts_exists_x,a3_proj_type,a3_st_bicycle_applies_x,a3_st_bicycle_pct_x,a3_st_num_schools_x,a3_st_ped_applies_x,a3_st_ped_pct_x,a3_st_srts_x,a3_trail_elig_cost_x,a3_trail_fed_funding_x,a3_trail_trans_pct_x,a3_trails_x,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode_x,a3_plan_none_x,a3_plan_other_x,a3_plan_other_desc_x,a2_output_outcome_x,a3_current_plan_x,b_sig_inter_new_bike_boxes_x,b_class_1_x,b_class_2_x,b_class_3_x,b_class_4_x,a4_bike_gap_pct_x,b_light_intersection_x,b_mid_block_new_rrfb_signal_x,b_mid_block_surf_improv_x,b_bsp_new_bikes_x,b_bike_new_secured_lockers_x,b_bike_new_racks_x,b_bsp_new_station_x,b_other_bike_improv_1_x,b_other_bike_improv_qty_1_x,b_other_bike_improv_2_x,b_other_bike_improv_qty_2_x,b_light_rdwy_seg_x,b_sig_inter_timing_improv_x,b_un_sig_new_rrfb_signal_x,b_un_sig_cross_surf_improv_x,a4_easement_support_x,m_cls_1_trails_widen_recon_exist_x,m_cls_1_trails_new__less_than_9_x,m_cls_1_trails_new_over_9_x,m_non_cls_trails_new_x,m_other_trail_imprv_1_x,m_other_trail_improv_qty_1_x,m_other_trail_imprv_2_x,m_other_trail_improv_qty_2_x,m_non_cls_widen_recon_exist_x,p_amenities_bench_x,a4_ped_gap_pct_x,p_mid_block_cross_new_rrfb_signal_x,p_light_intersection_x,p_lighting_rdwy_seg_x,p_mid_block_cross_surf_improv_x,p_new_ada_ramp_x,p_sidewlks_new_barrier_protect_x,p_sidewlks_new_4_to_8_x,p_sidewlks_new_over_8_x,p_other_ped_imprv_1_x,p_other_ped_qty_1_x,p_other_ped_imprv_2_x,p_other_ped_qty_2_x,p_reconstruct_ramp_to_ada_stand_x,p_sidewlks_reconstruct_enhance_exist_x,p_sig_inter_enhance_exist_crosswlk_x,p_sig_inter_new_crosswlk_x,p_sig_inter_ped_heads_x,p_sig_inter_shorten_cross_x,p_sig_inter_timing_improv_x,p_amenities_trash_can_x,p_amenities_shade_tree_x,p_amenities_shade_tree_type_x,p_un_sig_inter_new_traff_sig_x,p_un_sig_inter_new_roundabout_x,p_un_sig_inter_new_rrfb_sig_x,p_un_sig_inter_shorten_cross_x,p_un_sig_inter_cross_surface_improv_x,p_sidewlks_widen_existing_x,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1_x,v_speed_feedback_signs_x,v_other_traffic_calming_qty_1_x,v_other_traffic_calming_imprv_2_x,v_other_traffic_calming_qty_2_x,v_remove_right_turn_pocket_x,v_remove_travel_ln_x,v_sig_inter_new_roundabout_x,v_sig_inter_timing_improv_x,v_un_sig_inter_new_traf_sig_x,v_un_sig_inter_new_roundabout_x,app_fk,details_datetime_stamp,a4_reg_init_x,a4_reg_init_pct_x,a4_com_init_x,a4_com_init_pct_x,a4_safe_route_x,a4_safe_route_pct_x,a4_fl_mile_x,a4_fl_mile_pct_x,a4_emp_based_x,a4_emp_based_pct_x,a4_other_ni_x,a4_other_ni_descr_x,a4_other_ni_pct_x,a4_wb_audits_x,a4_bike_classes_x,a4_ped_classes_x,a4_demo_events_x,a4_com_enc_x,a4_le_methods_x,a4_com_meetings_x,a4_classrooms_x,a4_school_assem_x,a4_after_school_x,a4_bike_rodeos_x,a4_mock_cities_x,a4_walk_bus_x,a4_bike_train_x,a4_com_challenges_x,a4_srts_enc_x,a4_srts_le_x,a4_srts_training_x,a4_act_other_1_x,a4_act_other_1_descr_x,a4_act_other_2_x,a4_act_othe

In [58]:
dfall>>select(_.awarded_x, _.awarded_y,
               # _['#_x'], _['#_y'],
               # _.ppno_x, _.ppno_y,
               # _.ppno_1_x, _.ppno_1_y,
               _.a2_info_proj_descr_x, _.a2_info_proj_descr_y)>>filter(_.a2_info_proj_descr_y.notnull())

,awarded_x,awarded_y,a2_info_proj_descr_x,a2_info_proj_descr_y
8,N,Y,Design and construct buffered bike lanes on De...,Design and construct buffered bike lanes on De...
33,N,Y,This project focuses on school and pedestrian ...,This project focuses on school and pedestrian ...
35,N,Y,Construct Class IV separated bikeways with Cla...,Construct Class IV separated bikeways with Cla...
60,N,Y,Construction of .8 miles of Segment 7 of the R...,Construction of .8 miles of Segment 7 of the R...
66,N,Y,The Laurel Elementary SRTS includes infrastruc...,The Laurel Elementary SRTS includes infrastruc...
77,N,Y,Amador & Trinity and Church & Waldby intersect...,Amador & Trinity and Church & Waldby intersect...
81,N,Y,"Construct new curb, gutter, sidewalks, ADA ram...","Construct new curb, gutter, sidewalks, ADA ram..."
83,N,Y,"Completion of PS&E, ROW acquisition, and const...","Completion of PS&E, ROW acquisition, and const..."
91,N,Y,"In Happy Camp on SR 96, install sidewalks, con...","In Happy Camp on SR 96, install sidewalks, con..."
96,N,Y,"Construct 68 curb ramps, 87 crosswalks, advanc...","Construct 68 curb ramps, 87 crosswalks, advanc..."


In [59]:
compare_desc = np.where(dfall["a2_info_proj_descr_x"] == dfall["a2_info_proj_descr_y"], True, False)
dfall["compare_proj_desc"] = compare_desc

In [60]:
dfall.compare_proj_desc.value_counts()

False    837
True      47
Name: compare_proj_desc, dtype: int64

In [61]:
#check that there are no mismatched entries
dfall>>filter(_.compare_proj_desc==False)>>select(_.a2_info_proj_descr_x,  _.a2_info_proj_descr_y)>>arrange(_.a2_info_proj_descr_y)

,a2_info_proj_descr_x,a2_info_proj_descr_y
883,NaN,Neighborhood bike routes on four corridors in ...
882,NaN,The City of Maywood will develop an Active Tra...
0,"PA&ED, PS&E, and CON funding for construction ...",NaN
1,Bishop Street Class 3 Bicycle Boulevard with T...,NaN
2,CON funding for installing bicycling facilitie...,NaN
...,...,...
877,"Construction funding for Class IV bikelanes, ...",NaN
878,Construct Class II bike lane segments; install...,NaN
879,This project will decouple 2nd and 3rd street ...,NaN
880,"Curb extensions, sidewalks and crosswalks for ...",NaN


### test using concat

In [62]:
master_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 882 entries, 0 to 881
Columns: 218 entries, a1_imp_agcy_city to geometry
dtypes: Int64(9), datetime64[ns](2), float64(96), geometry(1), int64(16), object(94)
memory usage: 1.5+ MB


In [63]:
funded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Columns: 264 entries, awarded to data_origin
dtypes: float64(155), object(109)
memory usage: 101.4+ KB


In [64]:
dfall2 = (pd.concat([master_data, funded]))

In [65]:
dfall2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 931 entries, 0 to 48
Columns: 287 entries, a1_imp_agcy_city to year__con_ni_
dtypes: Float64(9), datetime64[ns](2), float64(150), geometry(1), object(125)
memory usage: 2.1+ MB


In [66]:
dfall2.sample()

,a1_imp_agcy_city,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_name,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_agcy,a1_proj_partner_exists,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_rout

In [67]:
len(dfall2>>group_by(_.project_app_id, _.project_cycle)>>summarize(n = _.awarded.nunique())>>arrange(_.n)>>filter(_.n>1))

47

In [68]:
dfall2>>group_by(_.project_app_id, _.project_cycle)>>summarize(n = _.awarded.nunique())>>arrange(_.n)>>filter(_.n>1)

,project_app_id,project_cycle,n
2,"1-Arcata, City of-1",5,2
3,"1-Clearlake, City of-1",5,2
36,10-Mariposa County-1,5,2
37,10-Mariposa County-2,5,2
77,10-Tuolumne County-1,5,2
100,"11-Imperial Beach, City of-1",5,2
112,"11-National City, City of-3",5,2
114,"11-Oceanside, City of-1",5,2
118,11-San Diego Association of Governments (SANDA...,5,2
223,2-Karuk Tribe-1,5,2


In [69]:
dfall2.awarded.value_counts()

N    882
Y     49
Name: awarded, dtype: int64

In [70]:
len(dfall2)

931

In [71]:
#dfall2['awarded'] = dfall2['awarded'].astype('category') 
# define the valid categories: 
#dfall2['awarded'] = dfall2['awarded'].cat.set_categories(['Y', 'N'], ordered=True) 

In [72]:
dfall2.sort_values(['project_app_id','a2_proj_scope_summary', 'project_cycle', 'awarded'], inplace=True, ascending=True) 

In [105]:
(dfall2>>select(_.project_app_id, _.a2_proj_scope_summary, _.project_cycle, _.awarded, _.data_origin)).head(50)

,project_app_id,a2_proj_scope_summary,project_cycle,awarded,data_origin
180,03-El Dorado County-1,The Project consists of a Class I multi-use gr...,5,N,Application
165,03-El Dorado County-2,The overall Diamond Springs Parkway project co...,5,N,Application
18,"1-Arcata, City of-1",The Arcata Annie & Mary Trail Connectivity Pro...,5.00,Y,Funded
175,"1-Arcata, City of-1",The Arcata Annie & Mary Trail Connectivity Pro...,5,N,Application
21,"1-Clearlake, City of-1",The project will expand bicycle and pedestrian...,5.00,Y,Funded
194,"1-Clearlake, City of-1",The project will expand bicycle and pedestrian...,5,N,Application
831,"1-Eureka, City of-1",The Bay to Zoo Trail creates approximately two...,6,N,Application
299,"1-Eureka, City of-1",The Bay to Zoo Trail creates approximately two...,5,N,Application
102,"1-Eureka, City of-2",C Street (between Waterfront Drive and Harris ...,5,N,Application
841,"1-Eureka, City of-2",C Street (between Waterfront Drive and Harris ...,6,N,Application


In [74]:
dfall3 = dfall2.sort_values('awarded').drop_duplicates(subset=['project_app_id','a2_proj_scope_summary','project_cycle'], keep='first')

In [107]:
dfall3>>select(_.project_app_id, _.a2_proj_scope_summary, _.project_cycle, _.awarded, _.data_origin)>>arrange(_.project_app_id)

,project_app_id,a2_proj_scope_summary,project_cycle,awarded,data_origin
180,03-El Dorado County-1,The Project consists of a Class I multi-use gr...,5,N,Application
165,03-El Dorado County-2,The overall Diamond Springs Parkway project co...,5,N,Application
18,"1-Arcata, City of-1",The Arcata Annie & Mary Trail Connectivity Pro...,5.00,Y,Funded
21,"1-Clearlake, City of-1",The project will expand bicycle and pedestrian...,5.00,Y,Funded
831,"1-Eureka, City of-1",The Bay to Zoo Trail creates approximately two...,6,N,Application
...,...,...,...,...,...
73,9-California Department of Transportation-6,This project will construct an ADA accessible ...,5,N,Application
168,"9-Tehachapi, City of-1",Dennison Road is one of three major north-sout...,5,N,Application
448,"9-Tehachapi, City of-1",This 0.5 mile stretch of Valley Boulevard serv...,6,N,Application
509,"9-Tehachapi, City of-2",The residential neighborhood north of Union Pa...,6,N,Application


In [108]:
(dfall3>>arrange(_.project_app_id, _.project_cycle)>>select(_.project_app_id, _.project_cycle, _.awarded, _.data_origin)).head(50)

,project_app_id,project_cycle,awarded,data_origin
180,03-El Dorado County-1,5,N,Application
165,03-El Dorado County-2,5,N,Application
18,"1-Arcata, City of-1",5.00,Y,Funded
21,"1-Clearlake, City of-1",5.00,Y,Funded
299,"1-Eureka, City of-1",5,N,Application
831,"1-Eureka, City of-1",6,N,Application
102,"1-Eureka, City of-2",5,N,Application
841,"1-Eureka, City of-2",6,N,Application
774,"1-Fortuna, City of-1",6,N,Application
267,1-Humboldt County Association of Governments-1,5,N,Application


In [77]:
#should have 882 - same number as the application data...
len(dfall3)

884

In [78]:
## the two that might show up twice may be from the changes in application name 
## same that did not merge in the first attempt
#dfall>>filter(_.full_merge=='right_only')>>select(_.project_app_id, _.a2_proj_scope_summary_x, _.project_cycle, _.awarded_x)>>arrange(_.project_app_id)

In [79]:
#dfall>>filter(_.project_app_id=='4-Oakland, City of-2')>>select(_.project_app_id,_.a1_imp_agcy_name_x, _.a1_imp_agcy_name_y, _.project_cycle)

In [80]:
dfall3>>group_by(_.project_app_id, _.project_cycle)>>summarize(n = _.awarded.nunique())>>arrange(_.n)

,project_app_id,project_cycle,n
0,03-El Dorado County-1,5.00,1
1,03-El Dorado County-2,5.00,1
2,"1-Arcata, City of-1",5.00,1
3,"1-Clearlake, City of-1",5.00,1
4,"1-Eureka, City of-1",5.00,1
...,...,...,...
879,9-California Department of Transportation-6,5.00,1
880,"9-Tehachapi, City of-1",5.00,1
881,"9-Tehachapi, City of-1",6.00,1
882,"9-Tehachapi, City of-2",5.00,1


In [81]:
dfall3.sample()

,a1_imp_agcy_city,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_name,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_agcy,a1_proj_partner_exists,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_proj_scope_summary,a2_project_location_map,a2_rtpa,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_collab_pub_health,a4_collab_le,a4_collab_non_profit,a4_collab_schools,a4_collab_pub_works,a4_collab_other,a4_colab_other_descr,a4_plan_ped,a4_plan_bike,a4_plan_atp,a4_plan_school_rout

In [82]:
# #unnamed cols originating in Funded data -- UPDATE NOW FIXED MANUALLY
# unnamed_cols = [col for col in funded.columns if isinstance(col, str) and re.match('unnamed.*', col)]
# unnamed_cols

## function for combining data - works

In [83]:
def join_funding_and_app_data(df_funding,
                              df_app,
                              awarded_col: list = [],
                             sort_values_cols: list = [],
                             subset_cols: list = []
                             ):
    '''
    columns in the funded and application data that we want to use
    awarded_col= ['awarded'],
    sort_values_cols = ['project_app_id','a2_proj_scope_summary', 'project_cycle', 'awarded'],
    subset_cols = ['project_app_id','a2_proj_scope_summary','project_cycle']
    '''
    # concat the funding and app dataframes
    df = (pd.concat([df_app, df_funding]))
    
    # take the awarded column and convert to a category so we can order by this column
    df[awarded_col] = df[awarded_col].astype('category') 
    df[awarded_col] = df[awarded_col].cat.set_categories(['Y', 'N'], ordered=True) 
    
    # sort values based on columns we defined (usually key like unique id, cycle)
    #df = df.sort_values(sort_values_cols, inplace=True, ascending=True) 
    
    # drop duplicates so we only get the funded data instead of the application data for a project that is selected
    df_final = df.sort_values(awarded_col).drop_duplicates(subset=subset_cols, keep='first')
    
    return df_final

In [84]:
df_test = join_funding_and_app_data(funded,
                              master_data,
                              awarded_col= 'awarded',
                             sort_values_cols= ['project_app_id','a2_proj_scope_summary', 'project_cycle', 'awarded'],
                             subset_cols = ['project_app_id','a2_proj_scope_summary','project_cycle']
                             )

In [85]:
df_test.sort_values(['project_app_id','a2_proj_scope_summary', 'project_cycle', 'awarded'], inplace=True, ascending=True) 

In [109]:
(df_test>>select(_.project_app_id, _.a2_proj_scope_summary, _.project_cycle, _.awarded, _.data_origin)).head(50)

,project_app_id,a2_proj_scope_summary,project_cycle,awarded,data_origin
180,03-El Dorado County-1,The Project consists of a Class I multi-use gr...,5,N,Application
165,03-El Dorado County-2,The overall Diamond Springs Parkway project co...,5,N,Application
18,"1-Arcata, City of-1",The Arcata Annie & Mary Trail Connectivity Pro...,5.00,Y,Funded
21,"1-Clearlake, City of-1",The project will expand bicycle and pedestrian...,5.00,Y,Funded
831,"1-Eureka, City of-1",The Bay to Zoo Trail creates approximately two...,6,N,Application
299,"1-Eureka, City of-1",The Bay to Zoo Trail creates approximately two...,5,N,Application
102,"1-Eureka, City of-2",C Street (between Waterfront Drive and Harris ...,5,N,Application
841,"1-Eureka, City of-2",C Street (between Waterfront Drive and Harris ...,6,N,Application
774,"1-Fortuna, City of-1",Pedestrian and bicycle facilities are intermit...,6,N,Application
267,1-Humboldt County Association of Governments-1,The purpose of HCAOG's Humboldt Regional Activ...,5,N,Application


In [111]:
df_test>>group_by(_.project_cycle)>>count(_.data_origin)

,project_cycle,data_origin,n
0,5,Application,401
1,5,Funded,49
2,6,Application,434


#### Check what columns are the duplicates

In [90]:
#https://stackoverflow.com/questions/61793094/find-column-whose-name-contains-a-specific-value-that-is-in-a-fixed-column

In [91]:
import re                                                               

In [92]:
dcolumns = [col for col in dfall.columns if isinstance(col, str) and re.match('.*_x', col)]

In [93]:
len(dcolumns)

196

In [94]:
dcolumns

['a1_imp_agcy_city_x',
 'a1_imp_agcy_fed_ma_num_x',
 'a1_imp_agcy_ma_x',
 'a1_imp_agcy_name_x',
 'a1_imp_agcy_state_ma_num_x',
 'a1_imp_agcy_street_x',
 'a1_imp_agcy_title_x',
 'a1_imp_agcy_zip_x',
 'a1_proj_partner_agcy_x',
 'a1_proj_partner_exists_x',
 'a1_proj_partner_title_x',
 'a2_assem_dist_a_x',
 'a2_assem_dist_b_x',
 'a2_assem_dist_c_x',
 'a2_congress_dist_a_x',
 'a2_congress_dist_b_x',
 'a2_congress_dist_c_x',
 'a2_county_x',
 'a2_info_proj_descr_x',
 'a2_info_proj_loc_x',
 'a2_info_proj_name_x',
 'a2_mop_uza_population_x',
 'a2_mpo_x',
 'a2_past_proj_x',
 'a2_past_proj_qty_x',
 'a2_proj_lat_x',
 'a2_proj_long_x',
 'a2_proj_scope_summary_x',
 'a2_project_location_map_x',
 'a2_rtpa_x',
 'a2_senate_dist_a_x',
 'a2_senate_dist_b_x',
 'a2_senatedistc_x',
 'a3_plan_active_trans_x',
 'a3_plan_active_trans_exists_x',
 'a3_plan_bicycle_x',
 'a3_plan_bicycle_exists_x',
 'a3_plan_ped_x',
 'a3_plan_ped_exists_x',
 'a3_plan_srts_x',
 'a3_plan_srts_exists_x',
 'a3_st_bicycle_applies_x',
 '

In [95]:
def remove_duplicate_cols(df, col_list):
    for col_x in col_list:
        df[col_x] = df[col_x].fillna(df[col_y])
    

In [96]:
dfall

,a1_imp_agcy_city_x,a1_imp_agcy_fed_ma_num_x,a1_imp_agcy_ma_x,a1_imp_agcy_name_x,a1_imp_agcy_state_ma_num_x,a1_imp_agcy_street_x,a1_imp_agcy_title_x,a1_imp_agcy_zip_x,a1_letter_of_intent,a1_proj_partner_agcy_x,a1_proj_partner_exists_x,a1_proj_partner_title_x,a2_assem_dist_a_x,a2_assem_dist_b_x,a2_assem_dist_c_x,a2_congress_dist_a_x,a2_congress_dist_b_x,a2_congress_dist_c_x,a2_county_x,a2_ct_dist,a2_info_proj_descr_x,a2_info_proj_loc_x,a2_info_proj_name_x,a2_mop_uza_population_x,a2_mpo_x,a2_past_proj_x,a2_past_proj_qty_x,a2_proj_lat_x,a2_proj_long_x,a2_proj_scope_summary_x,a2_project_location_map_x,a2_rtpa_x,a2_senate_dist_a_x,a2_senate_dist_b_x,a2_senatedistc_x,a3_plan_active_trans_x,a3_plan_active_trans_exists_x,a3_plan_bicycle_x,a3_plan_bicycle_exists_x,a3_plan_ped_x,a3_plan_ped_exists_x,a3_plan_srts_x,a3_plan_srts_exists_x,a3_proj_type,a3_st_bicycle_applies_x,a3_st_bicycle_pct_x,a3_st_num_schools_x,a3_st_ped_applies_x,a3_st_ped_pct_x,a3_st_srts_x,a3_trail_elig_cost_x,a3_trail_fed_funding_x,a3_trail_trans_pct_x,a3_trails_x,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,project_app_id,project_cycle,a1_locode_x,a3_plan_none_x,a3_plan_other_x,a3_plan_other_desc_x,a2_output_outcome_x,a3_current_plan_x,b_sig_inter_new_bike_boxes_x,b_class_1_x,b_class_2_x,b_class_3_x,b_class_4_x,a4_bike_gap_pct_x,b_light_intersection_x,b_mid_block_new_rrfb_signal_x,b_mid_block_surf_improv_x,b_bsp_new_bikes_x,b_bike_new_secured_lockers_x,b_bike_new_racks_x,b_bsp_new_station_x,b_other_bike_improv_1_x,b_other_bike_improv_qty_1_x,b_other_bike_improv_2_x,b_other_bike_improv_qty_2_x,b_light_rdwy_seg_x,b_sig_inter_timing_improv_x,b_un_sig_new_rrfb_signal_x,b_un_sig_cross_surf_improv_x,a4_easement_support_x,m_cls_1_trails_widen_recon_exist_x,m_cls_1_trails_new__less_than_9_x,m_cls_1_trails_new_over_9_x,m_non_cls_trails_new_x,m_other_trail_imprv_1_x,m_other_trail_improv_qty_1_x,m_other_trail_imprv_2_x,m_other_trail_improv_qty_2_x,m_non_cls_widen_recon_exist_x,p_amenities_bench_x,a4_ped_gap_pct_x,p_mid_block_cross_new_rrfb_signal_x,p_light_intersection_x,p_lighting_rdwy_seg_x,p_mid_block_cross_surf_improv_x,p_new_ada_ramp_x,p_sidewlks_new_barrier_protect_x,p_sidewlks_new_4_to_8_x,p_sidewlks_new_over_8_x,p_other_ped_imprv_1_x,p_other_ped_qty_1_x,p_other_ped_imprv_2_x,p_other_ped_qty_2_x,p_reconstruct_ramp_to_ada_stand_x,p_sidewlks_reconstruct_enhance_exist_x,p_sig_inter_enhance_exist_crosswlk_x,p_sig_inter_new_crosswlk_x,p_sig_inter_ped_heads_x,p_sig_inter_shorten_cross_x,p_sig_inter_timing_improv_x,p_amenities_trash_can_x,p_amenities_shade_tree_x,p_amenities_shade_tree_type_x,p_un_sig_inter_new_traff_sig_x,p_un_sig_inter_new_roundabout_x,p_un_sig_inter_new_rrfb_sig_x,p_un_sig_inter_shorten_cross_x,p_un_sig_inter_cross_surface_improv_x,p_sidewlks_widen_existing_x,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1_x,v_speed_feedback_signs_x,v_other_traffic_calming_qty_1_x,v_other_traffic_calming_imprv_2_x,v_other_traffic_calming_qty_2_x,v_remove_right_turn_pocket_x,v_remove_travel_ln_x,v_sig_inter_new_roundabout_x,v_sig_inter_timing_improv_x,v_un_sig_inter_new_traf_sig_x,v_un_sig_inter_new_roundabout_x,app_fk,details_datetime_stamp,a4_reg_init_x,a4_reg_init_pct_x,a4_com_init_x,a4_com_init_pct_x,a4_safe_route_x,a4_safe_route_pct_x,a4_fl_mile_x,a4_fl_mile_pct_x,a4_emp_based_x,a4_emp_based_pct_x,a4_other_ni_x,a4_other_ni_descr_x,a4_other_ni_pct_x,a4_wb_audits_x,a4_bike_classes_x,a4_ped_classes_x,a4_demo_events_x,a4_com_enc_x,a4_le_methods_x,a4_com_meetings_x,a4_classrooms_x,a4_school_assem_x,a4_after_school_x,a4_bike_rodeos_x,a4_mock_cities_x,a4_walk_bus_x,a4_bike_train_x,a4_com_challenges_x,a4_srts_enc_x,a4_srts_le_x,a4_srts_training_x,a4_act_other_1_x,a4_act_other_1_descr_x,a4_act_other_2_x,a4_act_othe

In [97]:
## list of all columns
#col_list = sorted(dfall.columns.to_list())

In [98]:
#col_list

In [99]:
#dfall.columns.get_loc("a3_current_plan_x")

In [100]:
# (dfall>>select(_.a3_proj_type, 
#                _.a3_proj_type_x,
#               _.a3_proj_type_y)).info()

remove cols: 
* a3_proj_type_x

In [101]:
(dfall>>select(_.a3_proj_type, 
               _.a3_proj_type_x,
              _.a3_proj_type_y))>>arrange(_.a3_proj_type_y)

,a3_proj_type,a3_proj_type_x,a3_proj_type_y
35,Infrastructure + NI - Large,Infrastructure + NI - Large,Combined (IF and NI)
60,Infrastructure + NI - Large,Infrastructure + NI - Large,Combined (IF and NI)
66,Infrastructure + NI - Small,Infrastructure + NI - Small,Combined (IF and NI)
77,Infrastructure + NI - Small,Infrastructure + NI - Small,Combined (IF and NI)
96,Infrastructure + NI - Small,Infrastructure + NI - Small,Combined (IF and NI)
...,...,...,...
877,Infrastructure - Small,NaN,NaN
878,Infrastructure - Small,NaN,NaN
879,Infrastructure - Large,NaN,NaN
880,Infrastructure + NI - Medium,NaN,NaN


In [102]:
#(dfall.iloc[:,64:70]).info()